In [2]:
import os
import sys
sys.path.insert(0,os.path.expanduser('~/git/torchseg'))
print(sys.path)

%reload_ext autoreload
%autoreload 2
from utils.summary_to_jupyter import summary,dump

['/data02/jiaxin15/git/torchseg', '', '/data02/jiaxin15/git/torchseg/notebooks', '/data02/jiaxin15/bin/miniconda3/envs/torch1.0/lib/python36.zip', '/data02/jiaxin15/bin/miniconda3/envs/torch1.0/lib/python3.6', '/data02/jiaxin15/bin/miniconda3/envs/torch1.0/lib/python3.6/lib-dynload', '/data02/jiaxin15/bin/miniconda3/envs/torch1.0/lib/python3.6/site-packages', '/data02/jiaxin15/git/gnu/distiller', '/data02/jiaxin15/bin/miniconda3/envs/torch1.0/lib/python3.6/site-packages/IPython/extensions', '/data02/jiaxin15/.ipython']


In [15]:
tags=['val/avg_f','val/fmeasure']
rootpath=os.path.expanduser('~/tmp/logs/motion')
# attention_type aux_backbone filter_feature
invalid_param_list=['dir','log_time','root_path',
                    'test_path', 'train_path', 'val_path',
                    'use_optical_flow','flow_backbone','ignore_outOfRoi','exception_value']
note_gtags=[['dataset','note'],['dataset','note']]
note_gtags=None
notes=['task29'+str(i) for i in range(6)]
# notes=['task289']

tasks=dump(tags=tags,rootpath=rootpath,notes=notes,descripe=['note','epoch'],
     invalid_param_list=invalid_param_list,delete_nan=False,dump_group=True,
     note_gtags=note_gtags)

/data02/jiaxin15/git/torchseg/utils/summary_to_jupyter.py:40: UserWarning: obtain dataset name from config file path
  ed['log_time']=log_time
/data02/jiaxin15/git/torchseg/utils/summary_to_jupyter.py:91: UserWarning: no group None column aux_backbone
  if len(col_set)>1 and (col not in invalid_param_list) and (col not in tags):


### task290 input_format,note,actual_step
|                               |   val/avg_f |   val/fmeasure |   epoch |
|:------------------------------|------------:|---------------:|--------:|
| ('-', 'task290_in-_attd', 50) |    0.73254  |       0.747131 |      50 |
| ('n', 'task290_inn_attd', 50) |    0.747061 |       0.758662 |      50 |
| ('-', 'task290_in-_attd', 46) |    0.771652 |       0.780226 |      50 |


|                               |   ('val/fmeasure', 'mean') |   ('val/fmeasure', 'std') |   ('val/fmeasure', 'amax') |
|:------------------------------|---------------------------:|--------------------------:|---------------------------:|
| ('-', 'task290_in-_attd', 46) |                   0.780226 |                       nan |                   0.780226 |
| ('-', 'task290_in-_attd', 50) |                   0.747131 |                       nan |                   0.747131 |
| ('n', 'task290_inn_attd', 50) |                   0.758662 |                       nan |           

/data02/jiaxin15/git/torchseg/utils/summary_to_jupyter.py:69: UserWarning: task294 task is empty
  


### task295 note,use_part_number,actual_step
|                             |   val/avg_f |   val/fmeasure |   epoch |
|:----------------------------|------------:|---------------:|--------:|
| ('task295_n1000', 1000, 48) |    0.761927 |       0.755595 |      50 |
| ('task295_n0', 0, 1)        |  nan        |     nan        |      50 |


|                             |   ('val/fmeasure', 'mean') |   ('val/fmeasure', 'std') |   ('val/fmeasure', 'amax') |
|:----------------------------|---------------------------:|--------------------------:|---------------------------:|
| ('task295_n0', 0, 1)        |                 nan        |                       nan |                 nan        |
| ('task295_n1000', 1000, 48) |                   0.755595 |                       nan |                   0.755595 |


|    | note          |   actual_step |   use_part_number |   val/avg_f |   val/fmeasure |   epoch |
|---:|:--------------|--------------:|------------------:|------------:|---------------

In [7]:
tags=['val/avg_f','val/fmeasure']
rootpath=os.path.expanduser('~/tmp/logs/motion')
notes=['today','recent']
note_gtags=[['dataset','log_time','net_name'],
           ['dataset','log_time','net_name']]
recent_log_number=100
tasks=dump(tags=tags,rootpath=rootpath,notes=notes,note_gtags=note_gtags,sort_tags=['dataset','val/fmeasure'],
     descripe=['note','epoch'],delete_nan=False,dump_group=False,recent_log_number=recent_log_number)

/data02/jiaxin15/git/torchseg/utils/summary_to_jupyter.py:71: UserWarning: today task is empty
  warnings.warn('{} task is empty'.format(note))


|    |   val/avg_f | net_name         | dataset   | note                |   val/fmeasure |   epoch | log_time              |
|---:|------------:|:-----------------|:----------|:--------------------|---------------:|--------:|:----------------------|
| 40 |    0.893974 | motion_attention | FBMS      | task273_scgh        |       0.49329  |      50 | 2019-09-16___23-47-19 |
| 55 |    0.994553 | motion_attention | FBMS      | task273_scgh        |       0.59163  |      50 | 2019-09-17___01-59-23 |
| 25 |    0.739764 | motion_attention | FBMS      | task273_scgh        |       0.785792 |      50 | 2019-09-16___21-35-44 |
| 23 |    0.756334 | motion_attention | FBMS      | task273_sch         |       0.787211 |      50 | 2019-09-16___21-14-18 |
| 69 |    0.931885 | motion_attention | FBMS      | task273_scgh        |       0.789506 |      50 | 2019-09-17___04-11-12 |
| 68 |    0.757345 | motion_attention | FBMS      | task272_h           |       0.789967 |      50 | 2019-09-17___03-57-43 |


In [7]:
from tabulate import tabulate
notes=['recent']
recent_log_number=10000
tasks=dump(tags=tags,rootpath=rootpath,notes=notes,note_gtags=note_gtags,sort_tags=['dataset','val/fmeasure'],
     descripe=['note','epoch'],delete_nan=False,dump_group=False,recent_log_number=recent_log_number)

# tasks=summary(rootpath,tags,note)
print(tags)
sort_tags=['dataset','val/fmeasure']
show_tags=['note']+sort_tags
tasks_dict={}

threshold={'FBMS':0.8,
         'cdnet2014':0.7,
         'all':0.7,
          'all2':0.7,
          'all3':0.6}
for d,v in threshold.items():
    tasks_dict[d]=tasks[tasks['dataset']==d][tasks[tags[1]]>v]
    print(tabulate(tasks_dict[d][show_tags].sort_values(sort_tags),tablefmt='pipe',headers='keys'))

/data02/jiaxin15/git/torchseg/utils/summary_to_jupyter.py:43: UserWarning: obtain dataset name from config file path
  warnings.warn('obtain dataset name from config file path')
/data02/jiaxin15/git/torchseg/utils/summary_to_jupyter.py:95: UserWarning: no group None column app
  warnings.warn('no group None column {}'.format(col))
/data02/jiaxin15/git/torchseg/utils/summary_to_jupyter.py:95: UserWarning: no group None column backbone_freeze
  warnings.warn('no group None column {}'.format(col))
/data02/jiaxin15/git/torchseg/utils/summary_to_jupyter.py:95: UserWarning: no group None column backbone_pretrained
  warnings.warn('no group None column {}'.format(col))
/data02/jiaxin15/git/torchseg/utils/summary_to_jupyter.py:99: UserWarning: exception in task
  warnings.warn('exception in task')


<class 'float'> input_shape
### recent backbone_name,dataset,epoch,frame_gap,freeze_layer,init_lr,net_name,note,pose_mask_reg,save_model,stn_loss_weight,upsample_layer,actual_step,dataset_name,use_part_number,ignore_outOfRoi,deconv_layer,use_none_layer,batch_size,freeze_ratio,sparse_ratio,upsample_type,psp_scale,use_aux_input,use_optical_flow,norm_stn_pose,subclass_sigmoid,flow_backbone,fusion_type,main_panet,share_backbone,decode_main_layer,min_channel_number,smooth_ratio,filter_type,aux_backbone,aux_freeze,optimizer,filter_feature,class_number,max_channel_number,attention_type,accumulate,ignore_pad_area,exception_value,loss_name,gpu,ngpus_per_node,rank,use_sync_bn,world_size,use_diff_img,input_format


/data02/jiaxin15/git/torchseg/utils/summary_to_jupyter.py:95: UserWarning: no group None column layer_preference
  warnings.warn('no group None column {}'.format(col))
/data02/jiaxin15/git/torchseg/utils/summary_to_jupyter.py:95: UserWarning: no group None column merge_type
  warnings.warn('no group None column {}'.format(col))
/data02/jiaxin15/git/torchseg/utils/summary_to_jupyter.py:95: UserWarning: no group None column modify_resnet_head
  warnings.warn('no group None column {}'.format(col))
/data02/jiaxin15/git/torchseg/utils/summary_to_jupyter.py:95: UserWarning: no group None column use_bias
  warnings.warn('no group None column {}'.format(col))
/data02/jiaxin15/git/torchseg/utils/summary_to_jupyter.py:95: UserWarning: no group None column use_bn
  warnings.warn('no group None column {}'.format(col))
/data02/jiaxin15/git/torchseg/utils/summary_to_jupyter.py:95: UserWarning: no group None column use_dropout
  warnings.warn('no group None column {}'.format(col))
/data02/jiaxin15/gi

|      |   val/fmeasure | dir                                                                                                                        |
|-----:|---------------:|:---------------------------------------------------------------------------------------------------------------------------|
| 1573 |     0          | /data02/jiaxin15/tmp/logs/motion/motion_panet2_stn/FBMS/task120_resnet50/2019-04-18___17-55-06/config.txt                  |
| 1119 |     0          | /data02/jiaxin15/tmp/logs/motion/motion_unet_flow/FBMS/task075/2019-04-11___01-59-09/config.txt                            |
| 1095 |     0          | /data02/jiaxin15/tmp/logs/motion/motion_panet2_flow/FBMS/task075/2019-04-10___21-33-43/config.txt                          |
| 1078 |     0          | /data02/jiaxin15/tmp/logs/motion/motion_panet2/FBMS/task073/2019-04-10___18-13-17/config.txt                               |
| 3348 |     0.00161107 | /data02/jiaxin15/tmp/logs/motion/motion_attention/FBMS/task236_motio

/data02/jiaxin15/bin/miniconda3/envs/torch1.0/lib/python3.6/site-packages/ipykernel_launcher.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


NameError: name 'tabulate' is not defined

In [9]:
from tabulate import tabulate
print(tabulate(tasks_dict[d][show_tags].sort_values(sort_tags),tablefmt='pipe',headers='keys'))

|      | note                          | dataset   |   val/fmeasure |
|-----:|:------------------------------|:----------|---------------:|
|  878 | task055_in288                 | FBMS      |       0.800045 |
|  774 | task051_first                 | FBMS      |       0.800089 |
|  672 | task043_subclass              | FBMS      |       0.800114 |
|  727 | task044_1.0                   | FBMS      |       0.800117 |
|  145 | task022_deconv                | FBMS      |       0.80018  |
|  890 | task055_in416                 | FBMS      |       0.80025  |
| 4133 | task286_in-_attq              | FBMS      |       0.800333 |
| 1474 | task109_MobileNetV2           | FBMS      |       0.80038  |
|  286 | task027_bs16                  | FBMS      |       0.800391 |
| 3544 | task259_scg                   | FBMS      |       0.800463 |
|  948 | task060                       | FBMS      |       0.800541 |
| 3507 | task259_gsc                   | FBMS      |       0.800598 |
| 1065 | task061    